<a href="https://colab.research.google.com/github/intrinio/intrinio-python-examples/blob/master/accessing_intrinio_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script uses your Intrinio Sandbox API Key. This key authorizes you to pull data for free for numerous data types like fundamentals and stock prices for DOW 30 companies. For production access to all companies, or all data sets, you will need a paid subscription to a specific data feed. 

In [0]:
#This installes the environment which includes your environment variables, passwords
!pip install colab-env -qU

#Installing the Intrinio Python SDK: https://docs.intrinio.com/documentation/python
!pip install intrinio-sdk


In [0]:
#Get packages we will need for this particular script
import colab_env
import pandas as pd
from psycopg2 import connect
import pandas.io.sql as psql
import os
from datetime import datetime
import numpy as np
import intrinio_sdk


In [0]:
#If this is your first time, you will need to create environment variables like this:
colab_env.envvar_handler.add_env("intrinio_sandbox_key", "YOUR_SANDBOX_KEY_HERE", overwrite=True)

#Make CERTAIN that you remove your API key and comment out or delete the line of code above after your run it. This ensures that your personal API key stays in your environment and doesn't get shared
#You only need to add your API key to your environment variable once in this script, but you will need to do it once per script.

#Note that you have multiple API keys in your Intrinio account. This script will work with a production key if you have a subscription with the correct license. If you don't have a subscription use your sandbox key

In [0]:
#Call your Intrinio api key from your Google Drive environment variable
intrinio_sandbox_key = os.getenv("intrinio_sandbox_key")

In [0]:
#Get Sandbox data, included for free with and Intrinio account: https://docs.intrinio.com/documentation/api_v2/sandbox, for Apple stock prices

from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

intrinio_sdk.ApiClient().configuration.api_key['api_key'] = intrinio_sandbox_key

security_api = intrinio_sdk.SecurityApi()

identifier = 'AAPL' # str | A Security identifier (Ticker, FIGI, ISIN, CUSIP, Intrinio ID)
start_date = '2018-01-01' # date | Return prices on or after the date (optional)
end_date = '2019-01-01' # date | Return prices on or before the date (optional)
frequency = 'daily' # str | Return stock prices in the given frequency (optional) (default to daily)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

try:
  api_response = security_api.get_security_stock_prices(identifier, start_date=start_date, end_date=end_date, frequency=frequency, page_size=page_size, next_page=next_page)
  #pprint(api_response)
except ApiException as e:
  print("Exception when calling SecurityApi->get_security_stock_prices: %s\r\n" % e)

In [0]:
#Assign the api response to a variable, inside of the Pandas Dataframe object
aapl_stock_prices = pd.DataFrame(api_response.stock_prices_dict)

In [0]:
#Print the api_response_df dataframe
aapl_stock_prices.head()

In [0]:
#Download a copy of the output 
from google.colab import files

aapl_stock_prices.to_csv('stock_prices.csv')
files.download('stock_prices.csv')

In [0]:
#Repeating the process to get all company filings for Apple. 

intrinio_sdk.ApiClient().configuration.api_key['api_key'] = intrinio_sandbox_key

company_api = intrinio_sdk.CompanyApi()

identifier = 'AAPL' # str | A Company identifier (Ticker, CIK, LEI, Intrinio ID)
filed_after = '' # date | Filed on or after this date (optional)
filed_before = '' # date | Filed on or before this date (optional)
reported_only = False # bool | Only as-reported fundamentals (optional)
fiscal_year = '' # int | Only for the given fiscal year (optional)
statement_code = '' # str | Only of the given statement code (optional)
type = '' # str | Only of the given type (optional)
start_date = '' # date | Only on or after the given date (optional)
end_date = '' # date | Only on or before the given date (optional)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

try:
  api_response_2 = company_api.get_company_fundamentals(identifier, filed_after=filed_after, filed_before=filed_before, reported_only=reported_only, fiscal_year=fiscal_year, statement_code=statement_code, type=type, start_date=start_date, end_date=end_date, page_size=page_size, next_page=next_page)
  #pprint(api_response)
except ApiException as e:
  print("Exception when calling CompanyApi->get_company_fundamentals: %s\r\n" % e)

In [0]:
#Converting to dataframe
all_company_filings = pd.DataFrame(api_response_2.fundamentals_dict)

#The filing id from this dataframe can be fed in a loop to retrieve all filings for a company.


In [0]:
#Pulling historical marketcap for AAPL

identifier = 'AAPL' # str | A Company identifier (Ticker, CIK, LEI, Intrinio ID)
tag = 'marketcap' # str | An Intrinio data tag ID or code reference [see - https://data.intrinio.com/data-tags]
frequency = 'daily' # str | Return historical data in the given frequency (optional) (default to daily)
type = '' # str | Return historical data for given fiscal period type (optional)
start_date = '2018-01-01' # date | Return historical data on or after this date (optional)
end_date = '' # date | Return historical data on or before this date (optional)
sort_order = 'desc' # str | Sort by date `asc` or `desc` (optional) (default to desc)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

try:
  api_response_3 = company_api.get_company_historical_data(identifier, tag, frequency=frequency, type=type, start_date=start_date, end_date=end_date, sort_order=sort_order, page_size=page_size, next_page=next_page)
  #pprint(api_response)
except ApiException as e:
  print("Exception when calling CompanyApi->get_company_historical_data: %s\r\n" % e)

In [0]:
#Converting to a dataframe
aapl_marketcap = pd.DataFrame(api_response_3.historical_data_dict)